In [ ]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
%cd "/content/drive/MyDrive/112_2_DL"
import os
os.listdir()

/content/drive/MyDrive/112_2_DL


['images', 'Assignment2_Q1.ipynb', 'Assignment2_Q2.ipynb']

In [1]:
import pandas as pd
import numpy as np
import cv2

In [2]:
train_set = pd.read_csv("images/train.txt", sep=" ", header=None)
valid_set = pd.read_csv("images/val.txt", sep=" ", header=None)
test_set = pd.read_csv("images/test.txt", sep=" ", header = None)

In [3]:
train_set = train_set.rename(columns = {0:"image", 1:"class"})
train_set

,image,class
0,images/n02111277/n02111277_10005.JPEG,0
1,images/n02111277/n02111277_10019.JPEG,0
2,images/n02111277/n02111277_10072.JPEG,0
3,images/n02111277/n02111277_1008.JPEG,0
4,images/n02111277/n02111277_10089.JPEG,0
...,...,...
63320,images/n02172182/n02172182_9515.JPEG,49
63321,images/n02172182/n02172182_954.JPEG,49
63322,images/n02172182/n02172182_9549.JPEG,49
63323,images/n02172182/n02172182_9589.JPEG,49


In [4]:
valid_set = valid_set.rename(columns = {0:"image", 1:"class"})
valid_set

,image,class
0,images/n02111277/n02111277_9695.JPEG,0
1,images/n02111277/n02111277_9747.JPEG,0
2,images/n02111277/n02111277_975.JPEG,0
3,images/n02111277/n02111277_9795.JPEG,0
4,images/n02111277/n02111277_980.JPEG,0
...,...,...
445,images/n02172182/n02172182_993.JPEG,49
446,images/n02172182/n02172182_9953.JPEG,49
447,images/n02172182/n02172182_997.JPEG,49
448,images/n02172182/n02172182_998.JPEG,49


In [5]:
test_set = test_set.rename(columns = {0:"image", 1:"class"})
test_set

,image,class
0,images/n02111277/n02111277_9420.JPEG,0
1,images/n02111277/n02111277_9422.JPEG,0
2,images/n02111277/n02111277_9484.JPEG,0
3,images/n02111277/n02111277_951.JPEG,0
4,images/n02111277/n02111277_9518.JPEG,0
...,...,...
445,images/n02172182/n02172182_974.JPEG,49
446,images/n02172182/n02172182_9765.JPEG,49
447,images/n02172182/n02172182_9789.JPEG,49
448,images/n02172182/n02172182_98.JPEG,49


In [6]:
train_img = []
for i in range(len(train_set["image"])):
    temp = cv2.imread("images/"+train_set["image"][i])
    temp = cv2.resize(temp, (128, 128))
    train_img.append(temp)

In [7]:
valid_img = []
for i in range(len(valid_set["image"])):
    temp = cv2.imread("images/"+valid_set["image"][i])
    temp = cv2.resize(temp, (128, 128))
    valid_img.append(temp)

In [8]:
test_img = []
for i in range(len(test_set["image"])):
    temp = cv2.imread("images/"+test_set["image"][i])
    temp = cv2.resize(temp, (128, 128))
    test_img.append(temp)

In [9]:
train_img = np.array(train_img)
valid_img = np.array(valid_img)
test_img = np.array(test_img)

In [10]:
import torch

In [11]:
x_train = torch.from_numpy(train_img)
x_valid = torch.from_numpy(valid_img)
x_test = torch.from_numpy(test_img)

In [12]:
x_train = x_train.permute(0,3,1,2)
x_valid = x_valid.permute(0,3,1,2)
x_test = x_test.permute(0,3,1,2)

In [13]:
y_train = train_set["class"]
y_valid = valid_set["class"]
y_test = test_set["class"]

In [14]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [15]:
class RRDB(nn.Module):
    def __init__(self, in_channels, growth_channels):
        super(RRDB, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, growth_channels, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(growth_channels, growth_channels, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(growth_channels, growth_channels, kernel_size=3, padding=1)
        #self.conv4 = nn.Conv2d(growth_channels, growth_channels, kernel_size=3, padding=1)
        self.conv5 = nn.Conv2d(growth_channels, in_channels, kernel_size=3, padding=1)

    def forward(self, x):
        residual = x
        out = F.leaky_relu(self.conv1(x))
        out = F.leaky_relu(self.conv2(out))
        out = F.leaky_relu(self.conv3(out))
        #out = F.leaky_relu(self.conv4(out))
        out = self.conv5(out)
        return residual + out

In [16]:
class Classification(nn.Module):
    def __init__(self, in_channels, num_classes):
        super(Classification, self).__init__()
        self.rrdb1 = RRDB(in_channels, 64)
        self.rrdb2 = RRDB(in_channels, 64)
        self.rrdb3 = RRDB(in_channels, 64)
        #self.rrdb4 = RRDB(in_channels, 64)  # 增加一個RRDB
        self.pool = nn.AdaptiveAvgPool2d(1)
        self.fc = nn.Linear(in_channels, num_classes)

    def forward(self, x):
        x = self.rrdb1(x)
        x = self.rrdb2(x)
        x = self.rrdb3(x)
        #x = self.rrdb4(x)  # 增加forward pass
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = self.fc(x)
        return x


In [17]:
# Model initialization
numclasses = 50
model = Classification(in_channels=3, num_classes=numclasses)  # Example for RGB images and 10 classes
print(model)

Classification(
  (rrdb1): RRDB(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (rrdb2): RRDB(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  )
  (rrdb3): RRDB(
    (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (conv5): Conv2d(64, 3, kernel_size=(3, 3), stride=(1, 1),

In [18]:
lr = 0.001
momentum = 0.9
weight_decay = 1e-4
epochs = 20
batch_size = 128
valid_batch_size = 20
test_batch_size = 20

In [19]:
import torch.optim as optim

device = 'cuda:0' if torch.cuda.is_available() else 'cpu'
# loader = DataLoader(list(zip(X_train, y_train)), shuffle=True, batch_size=16)
trainloader = torch.utils.data.DataLoader(list(zip(x_train, y_train)), batch_size=batch_size, shuffle=True)
validloader = torch.utils.data.DataLoader(list(zip(x_valid, y_valid)), batch_size=valid_batch_size, shuffle=True)
testloader = torch.utils.data.DataLoader(list(zip(x_test, y_test)), batch_size=test_batch_size, shuffle=False)
model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
#optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=weight_decay)

In [20]:
def train(epoch):
    model.train()
    avg_loss = 0.
    train_acc = 0.
    lr = 0.001
    for batch_idx, (data, target) in enumerate(trainloader):

        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data.to(torch.float32))
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()

        optimizer.step()
    print('Train Epoch: {}, loss{:.6f}, acc{}'.format(epoch, loss.item(), train_acc/len(trainloader.dataset)), end='')


In [21]:
def validation(epoch):
    model.eval()
    valid_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in validloader:
            data, label = data.to(device), label.to(device)
            output = model(data.to(torch.float32))
            valid_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    valid_loss/=len(validloader.dataset)
    correct = int(correct)
    print('Test set:average loss: {:.4f}, accuracy{}'.format(valid_loss, correct/len(validloader.dataset)))
    return correct/len(validloader.dataset)

In [22]:
best_val_acc=0.
val_acc = []
for i in range(epochs):
    train(i+1)
    temp_acc = validation(i+1)
    val_acc.append(temp_acc)
    if temp_acc > best_val_acc:
        best_val_acc = temp_acc
print('Best acc{}'.format(best_val_acc))

Train Epoch: 1, loss3.952657, acc0.020497433841228485

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Test set:average loss: 3.9350, accuracy0.015555555555555555
Train Epoch: 2, loss3.927357, acc0.021776549518108368Test set:average loss: 3.8980, accuracy0.02666666666666667


: 

In [ ]:
def test():
    model.eval()
    test_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(device), label.to(device)
            output = model(data.to(torch.float32))
            test_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    test_loss/=len(testloader.dataset)
    correct = int(correct)
    print('Test set:average loss: {:.4f}, accuracy{}'.format(test_loss, 100.*correct/len(testloader.dataset)))
    return correct/len(testloader.dataset)

In [ ]:
test()

In [22]:
from torchvision import models

model_resnet34 = models.resnet34(pretrained = False)
num_feature = model_resnet34.fc.in_features
model_resnet34.fc = torch.nn.Linear(num_feature, numclasses)
model_resnet34.to(device)
optimizer_resnet = optim.Adam(model_resnet34.parameters(), lr=lr)

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)


In [23]:
def train_resnet(epoch):
    model_resnet34.train()
    avg_loss = 0.
    train_acc = 0.
    for batch_idx, (data, target) in enumerate(trainloader):

        data, target = data.to(device), target.to(device)
        optimizer_resnet.zero_grad()
        output = model_resnet34(data.to(torch.float32))
        loss = F.cross_entropy(output, target)
        avg_loss += loss.item()
        pred = output.data.max(1, keepdim=True)[1]
        train_acc += pred.eq(target.data.view_as(pred)).cpu().sum()
        loss.backward()

        optimizer_resnet.step()
    print('Train Epoch: {}, loss{:.6f}, acc{}'.format(epoch, loss.item(), train_acc/len(trainloader.dataset)), end='')

In [24]:
def validation_resnet(epoch):
    model_resnet34.eval()
    valid_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in validloader:
            data, label = data.to(device), label.to(device)
            output = model_resnet34(data.to(torch.float32))
            valid_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    valid_loss/=len(validloader.dataset)
    correct = int(correct)
    print('Valid set:average loss: {:.4f}, accuracy{}'.format(valid_loss, correct/len(validloader.dataset)))
    return correct/len(validloader.dataset)

In [25]:
best_val_acc_34=0.
val_acc_34 = []
for i in range(epochs):
    train_resnet(i+1)
    temp_acc = validation_resnet(i+1)
    val_acc_34.append(temp_acc)
    if temp_acc > best_val_acc_34:
        best_val_acc_34 = temp_acc
print('Best acc{}'.format(best_val_acc_34))

Train Epoch: 1, loss2.731509, acc0.1607264131307602

/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/torch/nn/_reduction.py:42: UserWarning: size_average and reduce args will be deprecated, please use reduction='sum' instead.
  warnings.warn(warning.format(ret))


Valid set:average loss: 3.0240, accuracy0.19555555555555557
Train Epoch: 2, loss2.273692, acc0.2793525457382202Valid set:average loss: 2.8202, accuracy0.19777777777777777
Train Epoch: 3, loss2.193475, acc0.36268457770347595Valid set:average loss: 2.2272, accuracy0.34
Train Epoch: 4, loss1.936304, acc0.4371733069419861Valid set:average loss: 3.0605, accuracy0.28888888888888886
Train Epoch: 5, loss1.374545, acc0.5000710487365723Valid set:average loss: 2.0628, accuracy0.4177777777777778


In [ ]:
def test_resnet():
    model_resnet34.eval()
    test_loss = 0.
    correct=0.
    with torch.no_grad():
        for data, label in testloader:
            data, label = data.to(device), label.to(device)
            output = model_resnet34(data.to(torch.float32))
            test_loss += F.cross_entropy(output, label, size_average=False).item()
            pred =  output.data.max(1, keepdim=True)[1]
            correct += pred.eq(label.data.view_as(pred)).cpu().sum()
    test_loss/=len(testloader.dataset)
    correct = int(correct)
    print('Test set:average loss: {:.4f}, accuracy{}'.format(test_loss, 100.*correct/len(testloader.dataset)))
    return correct/len(testloader.dataset)

In [ ]:
test_resnet()